# Clustering Crypto

In [4]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from pathlib import Path

### Fetching Cryptocurrency Data

In [5]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [6]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.

In [18]:
# Alternatively, use the provided csv file:
file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame
crypto_df = pd.read_csv(file_path, index_col=0)
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000


### Data Preprocessing

In [19]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'


In [20]:
# Keep only cryptocurrencies that are trading
crypto_df.drop(crypto_df[crypto_df['IsTrading'] == False].index,
              inplace=True)

In [21]:
# Keep only cryptocurrencies with a working algorithm
crypto_df.drop(crypto_df[crypto_df['Algorithm'] == 'N/A'].index, inplace=True)

In [22]:
# Remove the "IsTrading" column
crypto_df.drop(columns = ['IsTrading'], inplace=True)

In [23]:
# Remove rows with at least 1 null value
crypto_df.dropna(inplace=True)

In [24]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df.drop(crypto_df[crypto_df['TotalCoinsMined'] == 0].index, inplace=True)

In [25]:
# Drop rows where there are 'N/A' text values
crypto_df = crypto_df[crypto_df != 'N/A'].dropna()

In [26]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
CoinName_df = crypto_df['CoinName'].to_frame()

In [27]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df.drop(columns = ['CoinName'], inplace=True)

In [28]:
# Create dummy variables for text features
X = pd.get_dummies(crypto_df, columns=['Algorithm', 'ProofType'], drop_first=True)

In [29]:
# Standardize data
X_scaled = StandardScaler().fit_transform(X)

### Reducing Dimensions Using PCA

In [30]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(X_scaled)

In [32]:
# Create a DataFrame with the principal components data
pca_df = pd.DataFrame(data=crypto_pca,
                            columns=['PC 1', 'PC 2', 'PC 3'],
                            index=crypto_df.index
                            )

pca_df

,PC 1,PC 2,PC 3
42,-0.340782,0.927509,-0.501594
404,-0.324112,0.927443,-0.501866
1337,2.312830,1.662479,-0.631417
BTC,-0.134016,-1.267036,0.174584
ETH,-0.137895,-1.932504,0.347313
...,...,...,...
ZEPH,2.487079,0.960044,-0.153524
GAP,-0.338826,0.927373,-0.501592
BDX,0.330910,-2.220182,0.422525
ZEN,-0.156906,-2.140278,0.469784


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [33]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=6)
    km.fit(pca_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow = {'k': k, 'inertia': inertia}
elbow_df = pd.DataFrame(elbow)
elbow_df.hvplot.line(x='k', y='inertia',
                     title='Elbow Curve',
                     xticks=k)

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [34]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)


# Fit the model
model.fit(pca_df)
# Predict clusters
predictions = model.predict(pca_df)
pca_df['class'] = model.labels_
# Create a new DataFrame including predicted clusters and cryptocurrencies features
clustered_df = pd.concat([crypto_df, CoinName_df, pca_df], axis=1)
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,CoinName,PC 1,PC 2,PC 3,class
42,Scrypt,PoW/PoS,4.199995e+01,42,42 Coin,-0.340782,0.927509,-0.501594,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,404Coin,-0.324112,0.927443,-0.501866,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,EliteCoin,2.312830,1.662479,-0.631417,0
BTC,SHA-256,PoW,1.792718e+07,21000000,Bitcoin,-0.134016,-1.267036,0.174584,3
ETH,Ethash,PoW,1.076842e+08,0,Ethereum,-0.137895,-1.932504,0.347313,3


### Visualizing Results

#### Scatter Plot with Tradable Cryptocurrencies

In [36]:
# Scale data to create the scatter plot
scaler = MinMaxScaler()

normalize_columns = ['TotalCoinsMined', 'TotalCoinSupply']
x = crypto_df[normalize_columns].values
x_scaled = scaler.fit_transform(x)

df_tradable = pd.DataFrame(x_scaled, columns=normalize_columns, index=crypto_df.index).reset_index()


In [37]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
df_tradable.hvplot.scatter(
    x='TotalCoinsMined',
    y='TotalCoinSupply',
    hover_cols=['index'],
)


:Scatter   [TotalCoinsMined]   (TotalCoinSupply,index)

#### Table of Tradable Cryptocurrencies

In [39]:
# Table with tradable cryptos
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType',
                                  'TotalCoinSupply', 'TotalCoinsMined',
                                  'class']
                         )

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,class]

In [38]:
# Print the total number of tradable cryptocurrencies
print(f'The total number of tradable cryptocurrenices is: {len(clustered_df)}')

The total number of tradable cryptocurrenices is: 533
